# Environement Setup

In [1]:
import sys, os
try:
    from google.colab import drive, userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

REPO_NAME = 'MistakeDetection'

if IS_COLAB:
    print("☁️ Colab rilevato.")
    if not os.path.exists('/content/drive'): drive.mount('/content/drive')

    GITHUB_USER = 'MarcoPernoVDP'
    try:
        TOKEN = userdata.get('GITHUB_TOKEN')
        REPO_URL = f'https://{TOKEN}@github.com/{GITHUB_USER}/{REPO_NAME}.git'
    except:
        REPO_URL = f'https://github.com/{GITHUB_USER}/{REPO_NAME}.git'

    ROOT_DIR = f'/content/{REPO_NAME}'
    if not os.path.exists(ROOT_DIR):
        !git clone {REPO_URL}
    else:
        %cd {ROOT_DIR}
        !git pull
        %cd /content


else:
    print("Ambiente locale rilevato.")
    ROOT_DIR = os.getcwd()
    while not os.path.exists(os.path.join(ROOT_DIR, '.gitignore')) and ROOT_DIR != os.path.dirname(ROOT_DIR):
        ROOT_DIR = os.path.dirname(ROOT_DIR)

if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)


☁️ Colab rilevato.
/content/MistakeDetection
Already up to date.
/content


# Dataset Setup

In [3]:
from utils import setup_project
# Ora puoi passare agli import del modello
from dataset.capitain_cook_4d_mlp_dataset import CaptainCook4DMLP_Dataset, DatasetSource
from models.BaselineV2_Transformer import BaselineV2_Transformer
from dataset.utils import SplitType

# Esegue: Setup Dati (unzip/copy), Login WandB, Setup Device
device = setup_project.initialize(ROOT_DIR)

# Import wandb
import wandb

Setup Progetto in: /content/MistakeDetection
source_path: /content/drive/MyDrive/MistakeDetection
Setup Dati da: /content/drive/MyDrive/MistakeDetection
Inizio setup dati...
   Sorgente: /content/drive/MyDrive/MistakeDetection
   Destinazione: /content/MistakeDetection/data
Estrazione ZIP: omnivore.zip...
Copia cartella: annotation_json...
Estrazione ZIP: slowfast.zip...
Estrazione ZIP: 3dresnet.zip...
Estrazione ZIP: x3d.zip...
Estrazione ZIP: omnivore_test.zip...
Estrazione ZIP: error_recognition_best.zip...
Estrazione ZIP: features.zip...
Estrazione ZIP: perceptionencoder.zip...
Estrazione ZIP: captain_cook_4d_gopro_resized_test.zip...
Estrazione ZIP: egovlp_test.zip...
Estrazione ZIP: hiero.zip...
✅ Setup completato! Dati pronti in: /content/MistakeDetection/data


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: s339450 (s339450-politecnico-di-torino) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


WandB Logged in.
Device: cuda


# Configuration

In [ ]:
# Configurazione esperimento
DATASET_SOURCE = DatasetSource.HIERO

config = {
    "dataset": "CaptainCook4D",
    "feature_extractor": DATASET_SOURCE.value,
    "input_dim": DATASET_SOURCE.input_dims(),
    "batch_size": 1,  # DEVE essere 1 per sequenze di lunghezza variabile
    "learning_rate": 1e-5,
    "epochs": 30,
    "pos_weight": 0.75,
    "optimizer": "Adam",
    "loss_function": "BCEWithLogitsLoss",
    "seed": 42,
    "type": "8_steps"
}

# Leave-One-Out Cross-Validation Setup

In [15]:
import os
from dataset.capitain_cook_4d_mlp_dataset import DatasetSource
from dataset.capitain_cook_4d_task2subtask2_dataset import CaptainCook4DTask2Subtask2_Dataset
from torch.utils.data import DataLoader, Subset
from collections import defaultdict

try:
    full_dataset = CaptainCook4DTask2Subtask2_Dataset(
        root_dir=ROOT_DIR
    )

    # Raggruppa i video per recipe (activity_id)
    # L'activity_id è il primo numero del video_id (es: "1_10" -> activity_id = 1)
    recipe_to_indices = defaultdict(list)
    for idx, video_id in enumerate(full_dataset.video_ids):
        activity_id = video_id.split('_')[0]
        recipe_to_indices[activity_id].append(idx)

    # Ordina le ricette per avere un ordine consistente
    recipes = sorted(recipe_to_indices.keys())

    print(f"\n{'='*80}")
    print(f"LEAVE-ONE-OUT CROSS-VALIDATION SETUP")
    print(f"{'='*80}")
    print(f"Total videos: {len(full_dataset)}")
    print(f"Total recipes (activities): {len(recipes)}")
    print(f"\nVideos per recipe:")
    for recipe_id in recipes:
        print(f"  Recipe {recipe_id}: {len(recipe_to_indices[recipe_id])} videos")
    print(f"{'='*80}\n")

except Exception as e:
    print(f"❌ Errore: {e}")

Loading from: /content/MistakeDetection/data/hiero...
Dataset creato: 384 video completi

LEAVE-ONE-OUT CROSS-VALIDATION SETUP
Total videos: 384
Total recipes (activities): 24

Videos per recipe:
  Recipe 1: 18 videos
  Recipe 10: 12 videos
  Recipe 12: 18 videos
  Recipe 13: 14 videos
  Recipe 15: 15 videos
  Recipe 16: 16 videos
  Recipe 17: 20 videos
  Recipe 18: 15 videos
  Recipe 2: 16 videos
  Recipe 20: 14 videos
  Recipe 21: 19 videos
  Recipe 22: 17 videos
  Recipe 23: 16 videos
  Recipe 25: 15 videos
  Recipe 26: 17 videos
  Recipe 27: 15 videos
  Recipe 28: 18 videos
  Recipe 29: 18 videos
  Recipe 3: 13 videos
  Recipe 4: 17 videos
  Recipe 5: 15 videos
  Recipe 7: 16 videos
  Recipe 8: 16 videos
  Recipe 9: 14 videos



In [6]:
# V2: quando accedi a dataset[idx], dove idx è l'indice dello STEP
full_dataset.print_item(0)

VIDEO DATASET ITEM [0]
Features shape:       torch.Size([8, 1024]) (num_steps, n_features)
Number of steps:      8
Label:                0 (No Errors)
Video ID:             10_16


In [16]:
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, roc_auc_score
import numpy as np
from models.BaselineV2_Transformer import BaselineV2_Transformer

# Inizializzazione W&B per l'intero esperimento LOO
run = wandb.init(
    project="mistake-detection",
    name=f"LOO-Task2Subtask2-{DATASET_SOURCE.value}",
    config=config,
    tags=["leave-one-out", "Task2Subtask2", DATASET_SOURCE.value],
    notes=f"Leave-One-Out CV with {DATASET_SOURCE.value} features for mistake detection"
)

print(f"🚀 W&B Run: {run.name} (ID: {run.id})")

# Statistiche per aggregare i risultati di tutti i fold
all_fold_results = []

# LOO: per ogni ricetta, usala come test set
for fold_idx, test_recipe_id in enumerate(recipes):
    print(f"\n{'='*80}")
    print(f"FOLD {fold_idx + 1}/{len(recipes)} - Testing on Recipe {test_recipe_id}")
    print(f"{'='*80}")

    # Indici del test set (ricetta corrente)
    test_indices = recipe_to_indices[test_recipe_id]

    # Indici del training set (tutte le altre ricette)
    train_indices = []
    for recipe_id in recipes:
        if recipe_id != test_recipe_id:
            train_indices.extend(recipe_to_indices[recipe_id])

    print(f"Train videos: {len(train_indices)} | Test videos: {len(test_indices)}")

    # Crea i subset
    train_dataset = Subset(full_dataset, train_indices)
    test_dataset = Subset(full_dataset, test_indices)

    # Crea i DataLoader
    train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False)

    # Inizializza un nuovo modello per questo fold
    model = BaselineV2_Transformer(DATASET_SOURCE.input_dims()).to(device)

    # Optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=config["learning_rate"])

    # Loss function con pos_weight
    train_pos_weight = torch.tensor([config["pos_weight"]], device=device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=train_pos_weight)

    # Training loop per questo fold
    best_train_loss = np.inf

    for epoch in range(config["epochs"]):
        # TRAIN
        model.train()
        total_loss = 0
        train_preds_list = []
        train_targets_list = []
        train_probs_list = []

        for inputs, labels, video_ids in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device).float()

            probs, logits = model(inputs)
            loss = criterion(logits, labels)
            total_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            with torch.no_grad():
                pred = (probs >= 0.5).long().item()
                train_preds_list.append(pred)
                train_targets_list.append(labels.item())
                train_probs_list.append(probs.item())

        avg_train_loss = total_loss / len(train_loader)

        # Metriche di training
        train_preds = np.array(train_preds_list)
        train_targets = np.array(train_targets_list)
        train_probs = np.array(train_probs_list)

        train_acc = accuracy_score(train_targets, train_preds)
        train_f1 = f1_score(train_targets, train_preds, zero_division=0)

        # Log su W&B per questo fold
        wandb.log({
            f"fold_{fold_idx+1}/train_loss": avg_train_loss,
            f"fold_{fold_idx+1}/train_accuracy": train_acc,
            f"fold_{fold_idx+1}/train_f1": train_f1,
            f"fold_{fold_idx+1}/epoch": epoch + 1
        })

        print(f"  Epoch {epoch+1}/{config['epochs']} - Train Loss: {avg_train_loss:.4f} - Train Acc: {train_acc:.4f} - Train F1: {train_f1:.4f}")

        # Salva il miglior modello per questo fold
        if avg_train_loss < best_train_loss:
            best_train_loss = avg_train_loss
            best_model_state = model.state_dict()

    # Carica il miglior modello
    model.load_state_dict(best_model_state)

    # TEST per questo fold
    model.eval()
    test_preds_list = []
    test_targets_list = []
    test_probs_list = []

    with torch.no_grad():
        for inputs, labels, video_ids in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device).float()

            probs, logits = model(inputs)
            pred = (probs >= 0.5).long().item()

            test_preds_list.append(pred)
            test_targets_list.append(labels.item())
            test_probs_list.append(probs.item())

    # Metriche di test per questo fold
    test_preds = np.array(test_preds_list)
    test_targets = np.array(test_targets_list)
    test_probs = np.array(test_probs_list)

    test_acc = accuracy_score(test_targets, test_preds)
    test_f1 = f1_score(test_targets, test_preds, zero_division=0)
    test_precision = precision_score(test_targets, test_preds, zero_division=0)
    test_recall = recall_score(test_targets, test_preds, zero_division=0)

    try:
        test_auc = roc_auc_score(test_targets, test_probs)
    except ValueError:
        test_auc = 0.0

    # Salva i risultati di questo fold
    fold_result = {
        'fold': fold_idx + 1,
        'test_recipe': test_recipe_id,
        'accuracy': test_acc,
        'f1': test_f1,
        'precision': test_precision,
        'recall': test_recall,
        'auc': test_auc,
        'test_targets': test_targets,
        'test_preds': test_preds
    }
    all_fold_results.append(fold_result)

    # Log su W&B
    wandb.log({
        f"fold_{fold_idx+1}/test_accuracy": test_acc,
        f"fold_{fold_idx+1}/test_f1": test_f1,
        f"fold_{fold_idx+1}/test_precision": test_precision,
        f"fold_{fold_idx+1}/test_recall": test_recall,
        f"fold_{fold_idx+1}/test_auc": test_auc,
    })

    print(f"\n  Test Results for Recipe {test_recipe_id}:")
    print(f"    Accuracy: {test_acc:.4f}")
    print(f"    F1: {test_f1:.4f}")
    print(f"    Precision: {test_precision:.4f}")
    print(f"    Recall: {test_recall:.4f}")
    print(f"    AUC: {test_auc:.4f}")


print(f"\n{'='*80}")
print("🎉 Leave-One-Out Cross-Validation completato!")
print(f"{'='*80}")

fold_1/epoch,▁▂▃▃▄▅▆▆▇█
fold_1/test_accuracy,▁
fold_1/test_auc,▁
fold_1/test_f1,▁
fold_1/test_precision,▁
fold_1/test_recall,▁
fold_1/train_accuracy,▁▃▄▄▆▅▆▇██
fold_1/train_f1,▁▃▄▄▆▅▆▇██
fold_1/train_loss,█▇▆▆▅▅▄▃▂▁
fold_10/epoch,▁▂▃▃▄▅▆▆▇█
+216,...


🚀 W&B Run: LOO-Task2Subtask2-hiero (ID: nw0a7kjg)

FOLD 1/24 - Testing on Recipe 1
Train videos: 366 | Test videos: 18
  Epoch 1/30 - Train Loss: 0.6062 - Train Acc: 0.5328 - Train F1: 0.5995
  Epoch 2/30 - Train Loss: 0.5849 - Train Acc: 0.5519 - Train F1: 0.5751
  Epoch 3/30 - Train Loss: 0.5616 - Train Acc: 0.6120 - Train F1: 0.6183
  Epoch 4/30 - Train Loss: 0.5510 - Train Acc: 0.6066 - Train F1: 0.6327
  Epoch 5/30 - Train Loss: 0.5354 - Train Acc: 0.6393 - Train F1: 0.6598
  Epoch 6/30 - Train Loss: 0.5015 - Train Acc: 0.6940 - Train F1: 0.7200
  Epoch 7/30 - Train Loss: 0.4808 - Train Acc: 0.6995 - Train F1: 0.7250
  Epoch 8/30 - Train Loss: 0.4651 - Train Acc: 0.7077 - Train F1: 0.7291
  Epoch 9/30 - Train Loss: 0.4356 - Train Acc: 0.7459 - Train F1: 0.7646
  Epoch 10/30 - Train Loss: 0.3960 - Train Acc: 0.7842 - Train F1: 0.8010
  Epoch 11/30 - Train Loss: 0.3676 - Train Acc: 0.8115 - Train F1: 0.8305
  Epoch 12/30 - Train Loss: 0.3219 - Train Acc: 0.8333 - Train F1: 0.8501
  

In [17]:
# Calcola le statistiche aggregate su tutti i fold
accuracies = [r['accuracy'] for r in all_fold_results]
f1_scores = [r['f1'] for r in all_fold_results]
precisions = [r['precision'] for r in all_fold_results]
recalls = [r['recall'] for r in all_fold_results]
aucs = [r['auc'] for r in all_fold_results]

# Medie e deviazioni standard
mean_acc = np.mean(accuracies)
std_acc = np.std(accuracies)
mean_f1 = np.mean(f1_scores)
std_f1 = np.std(f1_scores)
mean_precision = np.mean(precisions)
std_precision = np.std(precisions)
mean_recall = np.mean(recalls)
std_recall = np.std(recalls)
mean_auc = np.mean(aucs)
std_auc = np.std(aucs)

# Stampa i risultati aggregati
print(f"\n{'='*80}")
print("AGGREGATED RESULTS ACROSS ALL FOLDS")
print(f"{'='*80}")
print(f"\nMetric            | Mean      | Std Dev")
print(f"{'-'*80}")
print(f"Accuracy          | {mean_acc:.4f}    | {std_acc:.4f}")
print(f"F1 Score          | {mean_f1:.4f}    | {std_f1:.4f}")
print(f"Precision         | {mean_precision:.4f}    | {std_precision:.4f}")
print(f"Recall            | {mean_recall:.4f}    | {std_recall:.4f}")
print(f"AUC               | {mean_auc:.4f}    | {std_auc:.4f}")
print(f"{'='*80}")

# Stampa i risultati per ogni fold
print(f"\nRESULTS PER FOLD:")
print(f"{'-'*80}")
print(f"Fold | Recipe | Accuracy | F1       | Precision | Recall   | AUC")
print(f"{'-'*80}")
for result in all_fold_results:
    print(f"{result['fold']:<4} | {result['test_recipe']:<6} | {result['accuracy']:.4f}   | {result['f1']:.4f}   | {result['precision']:.4f}    | {result['recall']:.4f}   | {result['auc']:.4f}")
print(f"{'='*80}")

# Log delle metriche aggregate su W&B
wandb.log({
    "overall/mean_accuracy": mean_acc,
    "overall/std_accuracy": std_acc,
    "overall/mean_f1": mean_f1,
    "overall/std_f1": std_f1,
    "overall/mean_precision": mean_precision,
    "overall/std_precision": std_precision,
    "overall/mean_recall": mean_recall,
    "overall/std_recall": std_recall,
    "overall/mean_auc": mean_auc,
    "overall/std_auc": std_auc,
})

# Crea una tabella per W&B con i risultati per fold
fold_table_data = []
for result in all_fold_results:
    fold_table_data.append([
        result['fold'],
        result['test_recipe'],
        result['accuracy'],
        result['f1'],
        result['precision'],
        result['recall'],
        result['auc']
    ])

wandb.log({
    "fold_results_table": wandb.Table(
        columns=["Fold", "Test Recipe", "Accuracy", "F1", "Precision", "Recall", "AUC"],
        data=fold_table_data
    )
})

# Confusion Matrix aggregata (concatena tutti i target e le predizioni)
all_targets = np.concatenate([r['test_targets'] for r in all_fold_results])
all_preds = np.concatenate([r['test_preds'] for r in all_fold_results])

cm_overall = confusion_matrix(all_targets, all_preds)
print(f"\nOVERALL CONFUSION MATRIX:")
print(cm_overall)

wandb.log({
    "overall/confusion_matrix": wandb.plot.confusion_matrix(
        probs=None,
        y_true=all_targets,
        preds=all_preds,
        class_names=["No Error", "Error"]
    )
})


AGGREGATED RESULTS ACROSS ALL FOLDS

Metric            | Mean      | Std Dev
--------------------------------------------------------------------------------
Accuracy          | 0.5741    | 0.1277
F1 Score          | 0.5997    | 0.1686
Precision         | 0.6778    | 0.1911
Recall            | 0.6264    | 0.2416
AUC               | 0.6659    | 0.1609

RESULTS PER FOLD:
--------------------------------------------------------------------------------
Fold | Recipe | Accuracy | F1       | Precision | Recall   | AUC
--------------------------------------------------------------------------------
1    | 1      | 0.6111   | 0.7586   | 0.6875    | 0.8462   | 0.7385
2    | 10     | 0.8333   | 0.8750   | 0.8750    | 0.8750   | 0.8125
3    | 12     | 0.5000   | 0.6087   | 0.4375    | 1.0000   | 0.8701
4    | 13     | 0.5000   | 0.5882   | 0.6250    | 0.5556   | 0.5556
5    | 15     | 0.4667   | 0.5000   | 0.6667    | 0.4000   | 0.6000
6    | 16     | 0.5625   | 0.5333   | 0.8000    | 0.4000   |

In [18]:
# Chiudi il run di W&B
wandb.finish()
print("🏁 W&B run terminato")

fold_1/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
fold_1/test_accuracy,▁
fold_1/test_auc,▁
fold_1/test_f1,▁
fold_1/test_precision,▁
fold_1/test_recall,▁
fold_1/train_accuracy,▁▁▂▂▃▄▄▄▄▅▅▆▆▆▆▆▆▇▇▇█▇█▇██████
fold_1/train_f1,▁▁▂▂▂▃▄▄▄▅▅▆▆▆▆▆▆▇▇██▇█▇██████
fold_1/train_loss,██▇▇▇▇▆▆▆▅▅▄▄▄▃▃▃▂▃▂▂▂▂▂▁▁▁▁▁▁
fold_10/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
+216,...


🏁 W&B run terminato
